# Motivation

论文原文：[《xDeepFM: Combining Explicit and Implicit Feature Interactions for Recommender Systems》](https://arxiv.org/abs/1803.05170)

参考学习地址：[xDeepFM：名副其实的 ”Deep” Factorization Machine](https://zhuanlan.zhihu.com/p/57162373)

源代码：https://github.com/Leavingseason/xDeepFM

虽然xDeepFM和DeepFM名字接近，但和其最相关的实际是DCN。DeepFM思想比较直观，使用FM替换Wide部分的LR结构，另一个细节就是FM与Deep共享Embedding。DCN则使用Cross替换Wide部分的LR，实现自动显式的叉乘。

DCN 的 Cross Network 宣称能学习高阶交互，但其每一层的计算实际上是标量级的（x₀ * xₗᵀ * w），并且可以证明，它的输出是输入向量 x₀ 的标量倍数，表达能力有限，学到的交互模式本质上是特定形式的。

DCN以 **bit-wise** 方式自动叉乘，e.g. Age_Field嵌入向量<a1,b1,c1>，Occupation_Field嵌入向量<a2,b2,c2>，在Cross层，会拼接embedding得到<a1,b1,c1,a2,b2,c2>输入而丢失了Field vector概念。Cross以嵌入向量中的单个bit为最细粒度，而FM以向量为最细粒度（i.e. vector-wise）。**<font color="red">xDeepFM的动机，正是将FM的vector-wise的思想引入Cross部分</font>**。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/xdeepfm.png" width = "600" height = "400" alt="xDeepFM网络结构" align=center />

可以看到Wide和Deep结构能被保留，分别对应Linear和Plain DNN的部分。CIN部分为重点（和DNN共享Embedding，Linear直接使用原始输入）。
$$
\begin{align}
\hat{y}=\sigma(w_{linear}^Ta+w_{dnn}^Tx_{dnn}^k+w_{cin}^Tp^++b)
\end{align}
$$

 ## CIN：Compressed Interaction Network
 <font color="red">CIN本质上穷举了所有可能得到交叉结果，并给出w供模型记录学习结果。</font>
 * 交叉项应用vector-wise而非bit-wise
 * 显式高阶叉乘
 * 网络复杂度不随交叉程度指数增长

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/xdeepfm-cin.png" width = "1000" height = "400" alt="xDeepFM CIN网络结构" align=center />

 CIN层输入来自embeddingg层，假设有$m$个field，每个field的embeddingg vector维度为$D$，则输入可表示为矩阵$X^0\in R^{m\times D}$ 其中第 $i$ 行代表第 $i$ 个field vector $X_{0,*}^0=e_i$。第 $k$ 层CIN输出为 $X^k\in R^{H_k\times D}$，其中 $H_k$ 代表了该层对应的（embeddingg）特征向量数量，e.g. $H_0=m$（**Tips：$H_k$ 不一定等于 $H_{k-1}*m$，实际上 $H_k$ 是一个可以自由设定的超参数，通常设置为一个固定的常数或一个逐渐减小的序列**）。令$X^k\in R^{H_k*D}$表示第k层数出，其中$H_k$表示第k层的vector个数，vecor维度始终为D，和输入层一致，则第k层每个vector计算方式为：
$$
\begin{align}
X_{h,*}^k=\Sigma_{i=1}^{H_k-1}\Sigma_{j=1}^mW_{ij}^{k,h}(X_{i,*}^{k-1}\circ X_{j,*}^0),\quad \text{where } 1<=h<=H_k
\end{align}
$$
$W^{k,h}\in R^{H_{k-1}\times m}$代表第$h$个特征对应的参数矩阵，$\circ$代表Hadamard Product（也即element-wise乘积，e.g. $<a_1,a_2,a_3>\circ <b_1,b_2,b_3>=<a_1b_1,a_2b_2,a_3b_3>$，但注意CIN里的$a_1, b_1$实际为向量最后乘法得到笛卡尔叉乘矩阵）。公式详细的计算过程
1. 取前一层$X^{k-1}\in R^{H_{k01}*D}$中的$H_{k-1}$个vector，与输入层$X^0\in R^{m*D}$中的$m$个vecotr，进行两两hadamard乘积运算，得到$H_{k-1}*m$个vector，然后加权求和。
2. 第$k$层的不同vector区别在于，对这$H_{k-1}*m$个vector去和的权重矩阵不同， <font color="red"> **$H_k$ 即对应有多少个不同的权重矩阵$W^k$，是可调整的参数** </font>

CIN与DCN中Cross层的设计动机相似，Cross层的input也是前一层与输出层，原因是保持了DCN的优点（**有限高阶、自动叉乘、参数共享**）。主要的差异也有：
* Cross是bit-wise的，而CIN是vector-wise的
* 在第$l$层，**Cross包含1阶至l+1阶的所有组合特征，而CIN只包含l+1阶的组合特征**。相应的，Cross在输出层输出全部结果，而CIN在每层都输出中间结果。所以从网络结构上 **<font color="red">Cross本质上是ResNet，而CIN更接近RNN</font>**。

CIN也参考了CNN的结构，可以看到Hadamard乘积过程后的结果和卷积维度接近，而后的Sum Pooling对vector的元素加和输出。具体而言：
* 通过Hadamard乘积得到中间结果$Z^{k+1}\in R^{H_k*m*D}$
* 然后用权重矩阵$W^{k,i}\in R^{H_k*m}$顺着维度D，逐层相乘加和（i.e. Sum Pooling）得到k+1层的第i个vector。如果把W看作filter，和CNN方式很像，最后$Z^{k+1}$被压缩成了一个矩阵$x^{k+1}\in R^{H_{k+1}*D}$

设想CIN仅有一层，有m个vector（i.e. $H_1=m$），且加和的权重矩阵恒等于1（$W^1=1$），sum pooling的输出结果就是一系列的两两向量内积之和，也即标准的FM（不考虑一阶与偏置）。这种基于vector高阶组合也是其模型名称“eXtreme Deep Factorization Machine（xDeepFM）”的由来。

### 复杂度分析
假设CIN和DIN每层神经元/向量个数都为H，深度为T。那么CIN参数空间复杂度为$O(mTH^2)$，普通DNN为$O(mDH+TH^2)$，CIN空间复杂度与输入维度D无关。此外CIN可以通过对权重矩阵W进行矩阵分解从而降低空间复杂度。CIN的时间复杂度按上述计算方式为$O(mH^2DT)$而DNN为$O(mDH+TH^2)$，时间复杂度是CIN的主要痛点。

## 一些思考
Tips：xDeepFM中同时使用到了外积和哈达玛积，只不过应用在了不同阶段，区别参考DIN中。

先举个例子，假设 $m=2,D=2,H_k=3$，则有
$$
\begin{align*}
&X^0 = \begin{pmatrix} x_1 & x_2 \\ y_1 & y_2 \end{pmatrix}_{m \times D}
&&\text{,}
&& X^k = \begin{pmatrix} a_1 & a_2 \\ b_1 & b_2 \\ c_1 & c_2 \end{pmatrix}_{H_k \times D}
\end{align*}
$$
通过外积（Outer Product @，参考DIN）计算中间结果，用来表示生成所有可能的向量级（同语义）特征交互，（注：以下公式表达有误，理解含义即可）

$$
\begin{align*}
&Z^{k+1} = \begin{pmatrix} \begin{pmatrix} a_1 \\ b_1 \\ c_1 \end{pmatrix} \begin{pmatrix} x_1 & y_1 \end{pmatrix} & \begin{pmatrix} a_2 \\ b_2 \\ c_2 \end{pmatrix} \begin{pmatrix} x_2 & y_2 \end{pmatrix} 
\end{pmatrix}
= \begin{pmatrix} \begin{pmatrix} a_1x_1 & a_1y_1 \\ b_1x_1 & b_1y_1 \\ c_1x_1 & c_1y_1 \end{pmatrix}  & \begin{pmatrix} a_2x_2 & a_2y_2 \\ b_2x_2 & b_2y_2 \\ c_2x_2 & c_2y_2 \end{pmatrix}
\end{pmatrix}_{H_k\times m\times D}
\end{align*}
$$

假设，我们令 $H_{k+1}=4$，也即对应了4个不同的权重矩阵

$$
\begin{align*}
&W^{k+1,1} = \begin{pmatrix} w^1_{11} & w^1_{12} \\ w^1_{21} & w^1_{22} \\w^1_{31} & w^1_{32} \end{pmatrix}_{H_k \times m}
&&\text{,}
&W^{k+1,2} = \begin{pmatrix} w^2_{11} & w^2_{12} \\ w^2_{21} & w^2_{22} \\w^2_{31} & w^2_{32} \end{pmatrix}_{H_k \times m}
&&\text{,}
&W^{k+1,3} = \begin{pmatrix} w^3_{11} & w^3_{12} \\ w^3_{21} & w^3_{22} \\w^3_{31} & w^3_{32} \end{pmatrix}_{H_k \times m}
&&\text{,}
&W^{k+1,4} = \begin{pmatrix} w^4_{11} & w^4_{12} \\ w^4_{21} & w^4_{22} \\w^4_{31} & w^4_{32} \end{pmatrix}_{H_k \times m}
\end{align*}
$$

最终，使用Hadamard积（实际上结合了卷积）压缩这些交互得到

$$
\begin{align*}
X^{k+1} &= \begin{pmatrix} 
W^{k+1,1}\circ Z^{k+1,1} & W^{k+1,1}\circ Z^{k+1,2}\\ 
W^{k+1,2}\circ Z^{k+1,1} & W^{k+1,2}\circ Z^{k+1,2}\\ 
W^{k+1,3}\circ Z^{k+1,1} & W^{k+1,3}\circ Z^{k+1,2}\\ 
W^{k+1,4}\circ Z^{k+1,1} & W^{k+1,4}\circ Z^{k+1,2}
\end{pmatrix}= \begin{pmatrix} 
\begin{pmatrix} w^1_{11} & w^1_{12} \\ w^1_{21} & w^1_{22} \\w^1_{31} & w^1_{32} \end{pmatrix}_{H_k \times m}\circ \begin{pmatrix} a_1x_1 & a_1y_1 \\ b_1x_1 & b_1y_1 \\ c_1x_1 & c_1y_1 \end{pmatrix}_{H_k \times m} & \begin{pmatrix} w^1_{11} & w^1_{12} \\ w^1_{21} & w^1_{22} \\w^1_{31} & w^1_{32} \end{pmatrix}_{H_k \times m}\circ \begin{pmatrix} a_2x_2 & a_2y_2 \\ b_2x_2 & b_2y_2 \\ c_2x_2 & c_2y_2 \end{pmatrix}_{H_k \times m}\\ 
\begin{pmatrix} w^2_{11} & w^2_{12} \\ w^2_{21} & w^2_{22} \\w^2_{31} & w^2_{32} \end{pmatrix}_{H_k \times m}\circ \begin{pmatrix} a_1x_1 & a_1y_1 \\ b_1x_1 & b_1y_1 \\ c_1x_1 & c_1y_1 \end{pmatrix}_{H_k \times m} & \begin{pmatrix} w^2_{11} & w^2_{12} \\ w^2_{21} & w^2_{22} \\w^2_{31} & w^2_{32} \end{pmatrix}_{H_k \times m}\circ \begin{pmatrix} a_2x_2 & a_2y_2 \\ b_2x_2 & b_2y_2 \\ c_2x_2 & c_2y_2 \end{pmatrix}_{H_k \times m}\\ 
\begin{pmatrix} w^3_{11} & w^3_{12} \\ w^3_{21} & w^3_{22} \\w^3_{31} & w^3_{32} \end{pmatrix}_{H_k \times m}\circ \begin{pmatrix} a_1x_1 & a_1y_1 \\ b_1x_1 & b_1y_1 \\ c_1x_1 & c_1y_1 \end{pmatrix}_{H_k \times m} & \begin{pmatrix} w^3_{11} & w^3_{12} \\ w^3_{21} & w^3_{22} \\w^3_{31} & w^3_{32} \end{pmatrix}_{H_k \times m}\circ \begin{pmatrix} a_2x_2 & a_2y_2 \\ b_2x_2 & b_2y_2 \\ c_2x_2 & c_2y_2 \end{pmatrix}_{H_k \times m}\\ 
\begin{pmatrix} w^4_{11} & w^4_{12} \\ w^4_{21} & w^4_{22} \\w^4_{31} & w^4_{32} \end{pmatrix}_{H_k \times m}\circ \begin{pmatrix} a_1x_1 & a_1y_1 \\ b_1x_1 & b_1y_1 \\ c_1x_1 & c_1y_1 \end{pmatrix}_{H_k \times m} & \begin{pmatrix} w^4_{11} & w^4_{12} \\ w^4_{21} & w^4_{22} \\w^4_{31} & w^4_{32} \end{pmatrix}_{H_k \times m}\circ \begin{pmatrix} a_2x_2 & a_2y_2 \\ b_2x_2 & b_2y_2 \\ c_2x_2 & c_2y_2 \end{pmatrix}_{H_k \times m}
\end{pmatrix}_{H_{k+1} \times D}\\
&=\begin{pmatrix}
w^1_{11}a_1x_1+w^1_{12}a_1y_1+w^1_{21}b_1x_1+w^1_{22}b_1y_1+w^1_{31}c_1x_1+w^1_{32}c_1y_1 & w^1_{11}a_2x_2+w^1_{12}a_2y_2+w^1_{21}b_2x_2+w^1_{22}b_2y_2+w^1_{31}c_2x_2+w^1_{32}c_2y_2 \\
w^2_{11}a_1x_1+w^2_{12}a_1y_1+w^2_{21}b_1x_1+w^2_{22}b_1y_1+w^2_{31}c_1x_1+w^2_{32}c_1y_1 & w^2_{11}a_2x_2+w^2_{12}a_2y_2+w^2_{21}b_2x_2+w^2_{22}b_2y_2+w^2_{31}c_2x_2+w^2_{32}c_2y_2 \\
w^3_{11}a_1x_1+w^3_{12}a_1y_1+w^3_{21}b_1x_1+w^3_{22}b_1y_1+w^3_{31}c_1x_1+w^3_{32}c_1y_1 & w^3_{11}a_2x_2+w^3_{12}a_2y_2+w^3_{21}b_2x_2+w^3_{22}b_2y_2+w^3_{31}c_2x_2+w^3_{32}c_2y_2 \\
w^4_{11}a_1x_1+w^4_{12}a_1y_1+w^4_{21}b_1x_1+w^4_{22}b_1y_1+w^4_{31}c_1x_1+w^4_{32}c_1y_1 & w^4_{11}a_2x_2+w^4_{12}a_2y_2+w^4_{21}b_2x_2+w^4_{22}b_2y_2+w^4_{31}c_2x_2+w^4_{32}c_2y_2 \\
\end{pmatrix}_{H_{k+1} \times D}\\
\end{align*}
$$

另$T$代表CIN的网络深度，每一层CIN的输出需要对$X^{k}(k\in [1,T])$进行在维度$D$上的Sum Pooling：

$$
\begin{align*}
p^k=[p_1^k,p_2^k,...p_{H_k}^k]=[\Sigma_{j=1}^DX_{1,j}^k,\Sigma_{j=1}^DX_{2,j}^k,...,\Sigma_{j=1}^DX_{H_k,j}^k]
\end{align*}
$$
上例中：
$$
\begin{align*}
p^{k+1}&=\begin{pmatrix}
w^1_{11}a_1x_1+w^1_{12}a_1y_1+w^1_{21}b_1x_1+w^1_{22}b_1y_1+w^1_{31}c_1x_1+w^1_{32}c_1y_1 + w^1_{11}a_2x_2+w^1_{12}a_2y_2+w^1_{21}b_2x_2+w^1_{22}b_2y_2+w^1_{31}c_2x_2+w^1_{32}c_2y_2 \\
w^2_{11}a_1x_1+w^2_{12}a_1y_1+w^2_{21}b_1x_1+w^2_{22}b_1y_1+w^2_{31}c_1x_1+w^2_{32}c_1y_1 + w^2_{11}a_2x_2+w^2_{12}a_2y_2+w^2_{21}b_2x_2+w^2_{22}b_2y_2+w^2_{31}c_2x_2+w^2_{32}c_2y_2 \\
w^3_{11}a_1x_1+w^3_{12}a_1y_1+w^3_{21}b_1x_1+w^3_{22}b_1y_1+w^3_{31}c_1x_1+w^3_{32}c_1y_1 + w^3_{11}a_2x_2+w^3_{12}a_2y_2+w^3_{21}b_2x_2+w^3_{22}b_2y_2+w^3_{31}c_2x_2+w^3_{32}c_2y_2 \\
w^4_{11}a_1x_1+w^4_{12}a_1y_1+w^4_{21}b_1x_1+w^4_{22}b_1y_1+w^4_{31}c_1x_1+w^4_{32}c_1y_1 + w^4_{11}a_2x_2+w^4_{12}a_2y_2+w^4_{21}b_2x_2+w^4_{22}b_2y_2+w^4_{31}c_2x_2+w^4_{32}c_2y_2 \\
\end{pmatrix}_{H_{k+1} \times 1}\\
\end{align*}
$$

注意到，所有的$X^k$对应的维度都是相同的，每一层embedding_size相同，横向代表的是一个field。在计算$Z^{k+1}$时，使用同一维embedding的不同field进行笛卡尔交叉（i.e. vector-wise Hadamard Product），子项中都是下标相同但field不同的表示（e.g. $XX_1YY_1$），而不会出现不同下标的表示（e.g. $XX_1YY_2$）。而这就是CIN的核心哲学思想： <font color="red"> **！！！让同语义维度的特征交互，而不是让不同语义随意交叉** </font>。因为同一嵌入空间的维度可能编码表示特定语义（尽管不可解释），例如$d=1$可能表示“价格敏感度”，而$a_1*x_1$表示$X^k$和$X^0$向量的交叉价格敏感度。这样做的好处有：
* 语义对齐：通过哈达玛积实现向量维度的交互，从而表征交互项的语义对齐
* FM兼容：与FM向量级扩展一致，FM的二阶交互项$<v_i,v_j>=\Sigma_{d=1}^Dv_{i,d}*v_{j,d}$，CIN可以看作是FM的多层向量级扩展（FM对所有维度求和得到标量，CIN保持每个维度的交互结果，得到向量）。 <font color="red"> **设想CIN仅有一层，有m个vector（i.e. $H_1=m$），且加和的权重矩阵恒等于1（$W^1=1$），sum pooling的输出结果就是一系列的两两向量内积之和，也即标准的FM（不考虑一阶与偏置）** </font>
* 避免参数爆炸（压缩的表现）：若将所有可能的维度交叉（i.e.包含不同维），最终维度为$H_k\times m$，压缩后降维至$H_{k+1}$
* 不同维交叉：实际在最后Sum pooling的部分，通过线性变化也体现了部分跨维度的间接交互，例如：给$a_1x_1$赋予正权重，$a_2x_2$赋予副权重，虽$a_1$和$x_2$未直接相乘，但间接关系可以通过权重矩阵学习到。